In [ ]:
from numpy import False_
import pygame,sys
from random import randint, shuffle
from pygame.locals import QUIT

#遊戲初始化
pygame.init()

#遊戲視窗 #寬 1600 高 900
window = pygame.display.set_mode((1600, 900))
#遊戲標題
pygame.display.set_caption("PyGame")
#設定 文字格式
# Please use a unicode font. Change 'Arial Unicode MS.ttf' to any proper font file
font = pygame.font.Font('Arial Unicode MS.ttf', 16)
PATTERN_FONT = pygame.font.Font('Arial Unicode MS.ttf', 64)
PATTERN = ['♠','♣','♥','♦']
NUMBER = {1:"A",2:"2",3:"3",4:"4",5:"5",6:"6",7:"7",8:"8",9:"9",10:"10",11:"J",12:"Q",13:"K"}
CARD_WIDTH = 100
CARD_HEIGHT = 140
SPACING = 50
INTERVAL = 26
GREEN = (13, 139, 58)
GREY = (127, 127, 127)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

# 這是一張撲克牌
class Card:
    # 這是一張撲克牌
    def __init__(self,pattern, number, status):
        self.pattern = pattern
        self.number = number
        #牌是否覆蓋 以 True 代表翻開
        self.status = status
    # 回傳可渲染的 pygame.Surface 物件
    # focus 代表 是否被選取
    def blit(self, focus = 0):
        #### 可以用 window.blit(x.blit(), (x, y))
        #牌寬 100 高 140
        obj = pygame.Surface((CARD_WIDTH, CARD_HEIGHT))
        #代表 "空白牌" 綠色為底色
        if self.number == 0:
            obj.fill(GREEN)
            #可以直接中斷這樣就不用再往下執行
            return obj
        # 檢查是否為覆蓋狀態 以灰色帶表覆蓋
        # 代表說如果是覆蓋的牌就會直接 return 不再繼續執行
        elif not self.status :
            obj.fill(GREY)
        # 不然的話 以白色表示翻開
        else:
            obj.fill(WHITE)
            # 畫出圖案加數字
            face = self.pattern + NUMBER[self.number]
            #梅花黑桃畫黑色
            color = BLACK if self.pattern in ['♠','♣'] else RED
            obj.blit(font.render(face, True, color), (5, 0))

        # 如果是被選取的狀態 增加紅色邊框 
        #這個應該要挪出來寫，或是點擊之後判斷能不能選取
        if focus == 0:
            # 畫出 rect 指定你要畫的物件 並指定顏色 [x,y,寬,高], 厚度
            pygame.draw.rect(obj, BLACK, [0, 0, 100, 140], 3)
        else:
            pygame.draw.rect(obj, RED, [0, 0, 100, 140], 9)#有被選到有紅邊框

        return obj

class Pool:
    # 代表卡池，也就是維護剩下的卡片
    def __init__(self):
        # 初始化牌組
        #(pattern花色, number數字, False 牌蓋上)
        self.cards = [Card(pattern, number,status =  False) for pattern in PATTERN for number in NUMBER.keys()]
        #shuffle為隨機排序
        shuffle(self.cards)
        #右下角的四張底色 因為 number 帶 0 會代表底色牌
        self.RightButtomCornerPane = [Card(pattern = pattern ,number =  0, status = True) for pattern in PATTERN ]
    #牌堆剩多少牌
    #極富 Java 特色的函式
    def remaining(self):
        return len(self.cards)
    #發牌的動作
    def pop(self):
        #長度為 0 時代表沒牌可以發
        if len(self.cards):
            # list.pop() 會回傳他丟掉的牌
            return self.cards.pop()
        else:
            # 提供判斷用
            return False
#之後改個名字
class Pane:
    # 左下牌堆
    def __init__(self, pool):
        # 這裡直接讓他等於 pool 是因為在 Slot 那裏會呼叫 Pool.pop()這樣牌就會減少
        self.pool = pool
        #為了方便判斷 建立一個底部的牌
        # 0 代表 "空白牌" 綠色為底色
        self.cardDisclosed = [Card(pattern = "WithoutPattern",number = 0,status = True)]
        #代表選取狀態 一開始為不選取
        self.focus = False
    # 把牌移到右邊的牌堆 也就是 cardDisclosed
    def serve(self):    
        if self.pool.remaining() > 0: 
            self.cardDisclosed.append(self.pool.pop()) #最後一張拿掉放到首牌
        else:
            #self.hand[::-1]反轉, [:-1]只選取到最後一張之前
            #這樣 Card(0, 0, True) 就不會被選到
            self.pool.cards = self.cardDisclosed[::-1][:-1]
            for card in self.pool.cards:
                #左下未開牌 要再把它蓋回去
                card.status = False 
                # hand變回Card(0, 0, False)
                # 但因為必須要可以繼續 append 所以必須為 list 不能用 [0] 去替代
            self.cardDisclosed = self.cardDisclosed[:1]
        # 翻開最上面的牌
        self.cardDisclosed[-1].status = True
        # 選取狀態 設為 不選取
        self.setFocus(False)

    def setFocus(self, focus):
        # Focus 代表那張手牌正在被點選
        #以 > 1 判斷是否只有 Card(0, 0, True)
        self.focus = focus if len(self.cardDisclosed) > 1 else False

    def blit(self):
        # 左下牌區的總寬度跟長度
        obj = pygame.Surface((CARD_WIDTH * 2 + SPACING , CARD_HEIGHT))
        obj.fill(GREEN)
        # pool有牌才做的動作
        if self.pool.remaining() > 0:
            # 把 pool 最底下那張牌 畫在 obj 的範圍中 (0,0) 這個位置
            obj.blit(self.pool.cards[-1].blit(),(0,0))
            #畫 pool 的黑框   
            pygame.draw.rect(obj, BLACK, [0, 0, CARD_WIDTH, CARD_HEIGHT], 3)
            #渲染 pool 的文字 也就是剩下幾張牌的 X??
            txt = font.render("X" + str(self.pool.remaining()), True, BLACK)
            obj.blit(txt, (65, 120))
        # 畫 cardDisclosed 區域的牌
        obj.blit(self.cardDisclosed[-1].blit(self.focus),(CARD_WIDTH + SPACING, 0))
        return obj

class Slot:
    # 上面的單條 Slot
    # num 代表 Slot 的長度 cards 是他所擁有的牌
    # 傳入 pool 以提供牌
    
    def __init__(self, num, pool):
        # 初始化的時候抽 num 張牌
        # 抽 num 張牌
        self.cards = [pool.pop() for _ in range(num)]
        # 翻開最後一張牌
        self.cards[-1].status = True
        #代表選取狀態 一開始為不選取
        self.focus = False
        
    # 會回傳 slot 物件的高度，來判斷有沒有點擊到 slot    
    def height(self):
        #只有一張牌 ??????
        if len(self.cards) <= 1:
            return CARD_HEIGHT
        else:
            #疊加後的總高度
            return CARD_HEIGHT + (len(self.cards) - 1) * INTERVAL
    # 把 Slot 畫上去
    def blit(self):
        obj = pygame.Surface((CARD_WIDTH, CARD_HEIGHT + INTERVAL*18))
        obj.fill(GREEN)
        #畫沒牌時的黑框
        pygame.draw.rect(obj, BLACK, [0, 0, CARD_WIDTH, CARD_HEIGHT], 1)
        #間隔 INTERVAL 畫牌 沒有牌則不會跑迴圈
        for i, card in enumerate(self.cards):
            obj.blit(card.blit(), (0, INTERVAL * i))
            
        # 處理選到的牌以及其下面的牌 為他們加上紅框
        if self.focus:
            pygame.draw.rect(obj, RED, [0, self.index * INTERVAL, CARD_WIDTH, self.height() - self.index * INTERVAL], 6)
        return obj
    
    # 決定選取狀態
    def setFocus(self,index):
        # index 是指你選的那張牌的位置, 開頭為從 0 開始
        # 回顧 (y-80)//26 的回傳值
        self.index = index
        #下面這一步在呼叫 setFocus 之前就已經調整好了
        #self.index = min(self.index,len(self.cards)-1)

        #判斷何種情況 slot 處於 focus = True 的狀態
        if self.index < 0 or not self.cards[self.index].status: 
            self.focus = False
        else :
            self.focus = True
        #這裡應該是包含發牌的玩法
#         for i in range(self.index + 1, len(self.cards)):#選的是下張牌到slot最後一張
#             #這張牌和下張牌差不為1
#             if self.cards[i].number != self.cards[i - 1].number - 1:
#                 self.focus = -1
#                 return


class Place:
    def __init__(self, pattern):
        self.pattern = pattern
        self.cards = []

    def blit(self):
        obj = pygame.Surface((CARD_WIDTH, CARD_HEIGHT))
        obj.fill(GREEN)
        #畫沒牌時的黑框
        pygame.draw.rect(obj, BLACK, [0, 0, CARD_WIDTH, CARD_HEIGHT], 1)
        if len(self.cards) != 0:
            obj.blit(self.cards[-1].blit(),(0,0))
        else:
            txt = PATTERN_FONT.render(self.pattern,True, RED if self.pattern in ['♥','♦'] else BLACK)
            obj.blit(txt, (CARD_WIDTH/2 - 28, CARD_HEIGHT/2 - 42))
        return obj


class Game():
    def __init__(self):
        self.pool = Pool()
        #把牌從左到右的 slot 分一分
        self.slots = [Slot(i, self.pool) for i in range(1,8)]
        # 把剩下的牌給 pool
        self.pane = Pane(self.pool)
        #  四個Place
        self.places = [Place(pattern) for pattern in PATTERN]
        # 提供 上一次選取所選的牌
        self.chosenCard = []
        # 提供 上一次選取所選的區域
        self.chosenArea = None
        # 提供 上一次選取所選的 slot 的 index, 也就是第幾個 slot
        self.chosenSlotIndex = 0
        # 提供 上一次選取所選的 slot中, 你所選取的牌的 index, 也就是第幾張牌
        self.chosenSlot_CardIndex = 0
        
        # 因為有些參數 在不同情況不會給 所以有預設參數
        # 例如 選到 place 的時候 就不會傳入 chosenSlotIndex 跟 chosenSlot_CardIndex 參數
    def setFocusedArea(self,area,chosenSlotIndex = 0,chosenSlot_CardIndex = 0,pattern_Index = 0):
        # 最外圈的 if else 是判斷 area 的區域
        if area == "pane":
            # 呼叫 clean 是因為我會點擊 pane 就是為了拿牌存起來
            self.clean()
            self.pane.setFocus(True)
            self.chosenArea = area
            # 這裡將從 pane 拿的 card 存成 list 是為了判斷用
            self.chosenCard = self.pane.cardDisclosed[-1:]
################## 單個井字號會對應縮排 以代表是他下面那行的註解 ##################
################## 當我移到 slot 區或 place 區時 才要進行接牌的判定 ##################
##################接牌時的重點，"目前的選擇"以及"上一次的選擇"################## 
        # 當我移到 slot 區時
        elif area == "slot":
            # 檢查"上一次"的牌區是否為 slot 或 pane
            # 因為只有這兩個地方來的牌才能接在"目前的" slot 上面
            if self.chosenArea in ["slot","pane"]:
                # 先判斷"目前"接上的 slot 是否為空白的 
                if len(self.slots[chosenSlotIndex].cards) == 0 :
                    # 再判斷"上一次"儲存的牌, 也就是要接上去的牌, 最上面要等於 13
                    if self.chosenCard[0].number == 13:
                        # 使用 += 把"上一次"所存的牌接上"目前"所選的牌
                        self.slots[chosenSlotIndex].cards += self.chosenCard
                        # 如果"上一次"牌區在 slot 就要進行切掉牌的動作, 因為有可能為多張移動
                        if self.chosenArea == "slot":
                            # 將"上一次"所選的 slot 切掉"上一次"所儲存的牌
                            self.slots[self.chosenSlotIndex].cards = self.slots[self.chosenSlotIndex].cards[:self.chosenSlot_CardIndex]
                            # 切完牌之後決定要不要翻牌
                            # 如果"上一次"所選的 slot 切完後的第一張牌不是覆蓋的牌或是已經沒有牌了 那就不要翻牌
                            if len(self.slots[self.chosenSlotIndex].cards) > 0 and not self.slots[self.chosenSlotIndex].cards[-1].status:
                                self.slots[self.chosenSlotIndex].cards[-1].status = True
                            # 切牌跟翻牌執行完後代表要等待下一次的選取
                            # 所以先呼叫 clean 將上一次所存的牌 以及選取的狀態等等參數歸零
                            self.clean()
                        # 如果"上一次"牌區在 pane 則不用切 直接對 list 呼叫 pop 就好
                        else:
                            self.pane.cardDisclosed.pop()
                            # 跟上面一樣要等待下一次的選取
                            # 所以先呼叫 clean 將上一次所存的牌 以及選取的狀態等等參數歸零
                            self.clean()
                    # 如果"上一次"儲存的牌最上面不等於 13
                    # 就只是我點了一個背景區域 不用呼叫 clean, 等待下一次選取就好
                    else:
                        pass
                # 如果"目前"接上的 slot 不是空白的, 同時該牌要是翻開的狀態(註:我沒有避免點同個 slot 的情形)
                elif self.slots[chosenSlotIndex].cards[chosenSlot_CardIndex].status:
                    # 如果"上一次"儲存的牌最上面的牌 跟 "目前"的 slot 的最下面的牌 相差為 1
                    if self.chosenCard[0].number + 1 == self.slots[chosenSlotIndex].cards[-1].number:   
                        # 那就把"上一次"儲存的牌接上"目前"的 slot
                        self.slots[chosenSlotIndex].cards += self.chosenCard
                        # 跟上面一樣 如果"上一次"牌區在 slot 就要進行切掉牌的動作, 因為有可能為多張移動
                        if self.chosenArea == "slot":
                            # 將"上一次"所選的 slot 切掉"上一次"所儲存的牌
                            self.slots[self.chosenSlotIndex].cards = self.slots[self.chosenSlotIndex].cards[:self.chosenSlot_CardIndex]
                            # 切完牌之後決定要不要翻牌
                            # 如果"上一次"所選的 slot 切完後的第一張牌不是覆蓋的牌或是已經沒有牌了 那就不要翻牌
                            if len(self.slots[self.chosenSlotIndex].cards) > 0 and not self.slots[self.chosenSlotIndex].cards[-1].status:
                                self.slots[self.chosenSlotIndex].cards[-1].status = True
                            # 切牌跟翻牌執行完後代表要等待下一次的選取
                            # 所以先呼叫 clean 將上一次所存的牌 以及選取的狀態等等參數歸零
                            self.clean()
                        # 如果"上一次"牌區在 pane 則不用切 直接對 list 呼叫 pop 就好
                        else:
                            self.pane.cardDisclosed.pop()
                            # 跟上面一樣要等待下一次的選取
                            # 所以先呼叫 clean 將上一次所存的牌 以及選取的狀態等等參數歸零
                            self.clean()
                    # 代表不能接上"目前"選取的 slot, 那就把目前的 slot 資料存起來 
                    else:
                        # 先清空舊資料
                        self.clean()
                        # 將目前選取的資料保存 作為下一次的比較用
                        # 區域資料
                        self.chosenArea = area
                        # 目前所選的 slot 的 index
                        self.chosenSlotIndex = chosenSlotIndex
                        # 目前所選的牌的 index
                        self.chosenSlot_CardIndex = chosenSlot_CardIndex
                        # 目前所選的牌組            
                        self.chosenCard = self.slots[chosenSlotIndex].cards[chosenSlot_CardIndex:]
                        # 設定目前所選的 slot 為選取狀態
                        self.slots[chosenSlotIndex].setFocus(chosenSlot_CardIndex)
            # 代表選取的地方不在 slot 跟 pane 也就是空白處
            else:
                # 將目前選取的資料保存 作為下一次的比較用
                # 區域資料
                self.chosenArea = area
                # 目前所選的 slot 的 index
                self.chosenSlotIndex = chosenSlotIndex
                # 目前所選的牌的 index
                self.chosenSlot_CardIndex = chosenSlot_CardIndex
                # 目前所選的牌組
                self.chosenCard = self.slots[chosenSlotIndex].cards[chosenSlot_CardIndex:]
                # 設定目前所選的 slot 為選取狀態
                self.slots[chosenSlotIndex].setFocus(chosenSlot_CardIndex)
        #剩下就是 place 區域的判斷
        # place 區域因為不能把牌從這裡移出來只能移進去
        # 所以不會進行資料保存
        else:
            # 檢查上一次的牌區是否為 slot 或 pane
            # 因為只有這兩個地方來的牌才能接在 place 上面
            if self.chosenArea in ["slot","pane"]:
                # 先判斷"接上的" place 是否為空白的 
                if len(self.places[pattern_Index].cards) == 0 :
                    # 1.判斷"上一次"的牌最上面是否等於 1
                    # 2.判斷"上一次"的牌長度是否等於 1
                    # 3.判斷"上一次"的牌花色是否對應到"目前的" place 的花色
                    if len(self.chosenCard) == 1 and self.chosenCard[0].number == 1 and self.chosenCard[0].pattern == self.places[pattern_Index].pattern:   
                        # 都符合就把"上一次"的牌接上"目前的" place
                        self.places[pattern_Index].cards += self.chosenCard
                        # 如果"上一次"牌區在 slot 就要進行切掉牌的動作
                        # 這裡其實可以改成 pop 因為只會移動一張牌而已
                        if self.chosenArea == "slot":
                            self.slots[self.chosenSlotIndex].cards = self.slots[self.chosenSlotIndex].cards[:self.chosenSlot_CardIndex]
                            # 切完牌之後決定要不要翻牌
                            if len(self.slots[self.chosenSlotIndex].cards) > 0 and not self.slots[self.chosenSlotIndex].cards[-1].status:
                                self.slots[self.chosenSlotIndex].cards[-1].status = True
                            # 切牌跟翻牌執行完後代表要等待下一次的選取
                            # 所以先呼叫 clean 將上一次所存的牌 以及選取的狀態等等參數歸零
                            self.clean()
                        # 如果在 pane 則不用切 直接 pop 就好
                        else:
                            self.pane.cardDisclosed.pop()
                            # 跟上面一樣要等待下一次的選取
                            # 所以先呼叫 clean 將上一次所存的牌 以及選取的狀態等等參數歸零
                            self.clean()
                    # 如果沒有上面那三項判定沒有成功
                    # 就當作我點了一個背景區域 不用呼叫 clean, 等待下一次選取就好
                    else:
                        pass
                # 如果"目前"接上的 place 不是空白的
                else:
                    # 1.判斷"上一次"的牌最上面是否等於 1
                    # 2.判斷"上一次"的牌長度是否等於 1
                    # 3.判斷"上一次"的牌花色是否對應到"目前的" place 的花色
                    if len(self.chosenCard) == 1 and self.chosenCard[0].number - 1 == self.places[pattern_Index].cards[-1].number and self.chosenCard[0].pattern == self.places[pattern_Index].pattern:   
                        # 那就把"上一次"儲存的牌接上"目前"的 place
                        self.places[pattern_Index].cards += self.chosenCard
                        # 跟上面一樣 如果"上一次"牌區在 slot 就要進行切掉牌的動作
                        # 這裡其實可以改成 pop 因為只會移動一張牌而已
                        if self.chosenArea == "slot":
                            # 將"上一次"所選的 slot 切掉"上一次"所儲存的牌
                            self.slots[self.chosenSlotIndex].cards = self.slots[self.chosenSlotIndex].cards[:self.chosenSlot_CardIndex]
                            # 切完牌之後決定要不要翻牌
                            if len(self.slots[self.chosenSlotIndex].cards) > 0 and not self.slots[self.chosenSlotIndex].cards[-1].status:
                                self.slots[self.chosenSlotIndex].cards[-1].status = True
                            # 切牌跟翻牌執行完後代表要等待下一次的選取
                            # 所以先呼叫 clean 將上一次所存的牌 以及選取的狀態等等參數歸零
                            self.clean()
                        # 如果"上一次"牌區在 pane 則不用切 直接對 list 呼叫 pop 就好
                        else:
                            self.pane.cardDisclosed.pop()
                            # 跟上面一樣要等待下一次的選取
                            # 所以先呼叫 clean 將上一次所存的牌 以及選取的狀態等等參數歸零
                            self.clean()
                    # 如果沒有上面那三項判定沒有成功
                    # 就當作我點了一個背景區域 不用呼叫 clean, 等待下一次選取就好
                    else:
                        pass
            # 代表選取的地方不在 slot 跟 pane 也就是空白處
            # 不用呼叫 clean, 等待下一次選取就好
            else:
                pass
    # 當我點擊空白處時 要取消所有 Area 的選取,以及原先儲存的資料
    def clean(self):
        # 取消 pane 跟 "上一次"所選的 slot 的選取狀態
        self.pane.setFocus(False)
        self.slots[self.chosenSlotIndex].focus = False
        # 清空卡槽
        self.chosenCard = []
        # 參數歸零
        self.chosenSlotIndex = 0
        self.chosenArea = None
    #畫出全部畫面    
    def blit(self):
        # 建立物件
        obj = pygame.Surface((1600, 900))
        #填上綠色
        obj.fill(GREEN)
        # 畫上 pane
        obj.blit(self.pane.blit(), (100, 700))
        # 畫上 slots
        for y in range(len(self.slots)):
            obj.blit(self.slots[y].blit(), (300 + y * 150, 80))
        # 畫上 places
        for z in range(4):
            obj.blit(self.places[z].blit(), (750 + z * 150, 700))
        return obj        
#### --- 進入遊戲 --- ####


game = Game()

while True:

    window.blit(game.blit(), (0, 0))#畫面起始位置
    pygame.display.flip()

    for e in pygame.event.get():
        
        # 按下 X 離開遊戲
        if e.type == QUIT:
            pygame.quit()
            sys.exit()
            
        # 點擊畫面的事件
        if e.type == pygame.MOUSEBUTTONDOWN:
            #獲取滑鼠座標

            x, y = pygame.mouse.get_pos()
            flg = False
            
            # 點擊左下角發牌處
            if 100 <= x <= 200 and 700 <= y <= 840:
                game.pane.serve()
                flg = True
                
            # 點擊左下角"放"牌處
            if 250 <= x <= 250 + CARD_WIDTH and 700 <= y <= 840:
                # 呼叫 game.setFocusedArea() 代表我點擊了左下角"放"牌處的牌
                if len(game.pane.cardDisclosed) > 1 :
                    game.setFocusedArea(area ="pane")
                flg = True
            # 點擊某個 slot 處
            for i in range(7):
                # 先判斷是否在 slot 的範圍 以及 在該 slot 的長度範圍
                if 300 + i * 150 <= x <= 300 + i * 150 + 100 and 80 <= y <= 80 + game.slots[i].height():
                    # chosenSlotIndex 代表你選的 slot 是第幾個 slot
                    # 因為 game 裡面有 7 個 slot, 順序從 0 開始
                    # chosenSlot_CardIndex 為 他是那個 slot 中的第幾張牌, 一樣順序從 0 開始
                    game.setFocusedArea(area = "slot",chosenSlotIndex = i,chosenSlot_CardIndex = min((y - 80) // 26,len(game.slots[i].cards)-1))
                    flg = True                      
            # 點擊某個 place 處
            for i in range(4):
                # pattern_Index 代表第幾個 place, 一樣順序從 0 開始
                if 750 + i * 150 <= x <= 750 + i * 150 + CARD_WIDTH and 700 <= y <= 700 + CARD_HEIGHT :
                    game.setFocusedArea(area ="place",pattern_Index = i)
                    flg = True
            # 如果點擊背景，就把舊的紀錄清除掉
            if not flg:
                game.clean()
                
                